In [ ]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.colors as mpl_colors

from scipy import interpolate
import FlowCytometryTools as fct

In [ ]:
%load_ext autoreload
%autoreload 2
%aimport -fct -np -pd -plt

In [ ]:
sys.path.append('../')
from pipeline_process import common
from pipeline_process.facs import constants, manager, processor, unmixer

FITC = constants.FITC
BOX_ROOT = '/Users/keith.cheveralls/Box-cache/'

In [ ]:
# initialize a manager
m = manager.FACSManager(BOX_ROOT)

In [ ]:
# initialize a processor for one plate
p = processor.FACSProcessor(*m.get_sample_and_control_dirpaths('P0017'))

In [ ]:
# process a sample
plt.figure(figsize=(8,6))
s = p.process_sample('A11', show_plots=True)

In [ ]:
all_plate_ids = list(manager.CONTROL_DIRNAMES.keys())

### Process all plates

In [ ]:
# load all plates and process all samples
stats = []
for plate_id in all_plate_ids:
    print('Loading plate %s' % plate_id)
    p = processor.FACSProcessor(*m.get_sample_and_control_dirpaths(plate_id))

    for well_id in p.well_ids:
        stat = p.process_sample(well_id, show_plots=False)
        stat['well_id'] = well_id
        stat['plate_id'] = plate_id
        stats.append(stat)

In [ ]:
# cache results
pd.DataFrame(data=stats).to_csv('2019-07-16_all-facs-results.csv')

### Some plots

In [ ]:
d = pd.DataFrame(data=stats)
d = d.dropna(how='any', axis=0)
d.shape

In [ ]:
_ = plt.scatter(d.area, d.median_log, alpha=.1)

In [ ]:
sns.palplot(sns.color_palette())

In [ ]:
# area vs log median
_ = plt.hist2d(
    d.area,
    d.median_log,
    bins=(np.arange(0, 40, 1), np.arange(0, 1.5, .03)),
    norm=mpl_colors.PowerNorm(.5))

In [ ]:
# mean vs std
_ = plt.hist2d(
    d['mean'],
    d['std'],
    bins=(np.arange(3000, 7000, 90), np.arange(0, 1500, 30)),
    norm=mpl_colors.PowerNorm(.7))

In [ ]:
# std vs max
_ = plt.hist2d(
    d.intensity_mean.loc[mask],
    d.intensity_p99.loc[mask],
    bins=(np.arange(3000, 7000, 90), np.arange(4000, 10000, 60)),
    norm=mpl_colors.PowerNorm(.7))

### Compare control distributions from different plates

In [ ]:
# plot the reference distributions
plate_nums_weird_controls = [1, 6, 10, 11, 19]
for plate_num in plate_nums_weird_controls:
    print('Loading plate %s' % plate_num)
    p = processor.FACSProcessor(*m.sample_and_control_dirpaths(plate_num))
    plt.plot(p.x_ref, p.y_ref, label=plate_num)
plt.legend()